# 📉 Phase 4 : Évaluation & Robustesse

**Objectifs** :
1. Évaluer le système hybride sur le jeu de test.
2. Analyser les erreurs (Matrices de confusion).
3. Tester la **robustesse** face au bruit (simulation de trafic réel imparfait).
4. Comparer les performances par classe d'attaque.

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import sys
import os

# Ajouter le dossier src au path pour importer nos modules
sys.path.append(os.path.abspath('../../'))
from src.hybrid_ids import HybridIDS

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

plt.style.use('seaborn-v0_8-darkgrid')


## 1. Chargement des Données et du Système

In [ ]:
# 1. Charger le système hybride entraîné (Phase 3)
try:
    ids = HybridIDS.load('../../models/hybrid_ids_system.pkl')
    print("✅ Système Hybride chargé avec succès.")
except FileNotFoundError:
    print("❌ Erreur : Le modèle n'est pas trouvé. Avez-vous exécuté le notebook de Phase 3 ?")
    # En cas d'erreur, on arrête l'exécution ici si c'était un script, mais en notebook on continue pour l'exemple

# 2. Charger les données de test (Sauvegardées en Phase 2 ou 3)
# Pour cet exemple, on va recharger le CSV nettoyé et refaire le split pour garantir la cohérence
# Idéalement, on aurait sauvegardé X_test.npy et y_test.npy
df = pd.read_csv('../../data/processed/cicids2017_cleaned.csv')

# Recréer les labels (Comme en Phase 3)
df['Binary_Label'] = (df['Attack Type'] != 'Normal Traffic').astype(int)
df['Attack_Merged'] = df['Attack Type'].replace({'DoS': 'DoS_DDoS', 'DDoS': 'DoS_DDoS'})

# Séparer Features
label_cols = ['Attack Type', 'Binary_Label', 'Attack_Merged', 'Multiclass_Label'] 
# Note: Multiclass_Label n'existe peut-être pas dans le CSV si on ne l'a pas sauvegardé, on l'ignore
feature_cols = [col for col in df.columns if col not in label_cols and col not in ['Multiclass_Label']]

X = df[feature_cols]
y_true_type = df['Attack Type'] # Vrai type pour comparaison finale

# Prendre un échantillon de test (simuler le Test Set)
# Dans un vrai projet, on charge X_test sauvegardé pour ne pas tricher
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split(X, y_true_type, test_size=0.2, random_state=42, stratify=y_true_type)

print(f"Données de test prêtes : {X_test.shape}")

## 2. Évaluation Standard

In [ ]:
print("🚀 Lancement des prédictions sur le jeu de test...")
results = ids.predict_df(X_test)

# Ajouter les vrais labels pour comparer
results['True_Label'] = y_test.values

results.head()

In [ ]:
# Mapping pour aligner les noms (Le modèle a peut-être fusionné DoS/DDoS)
# On doit s'assurer que les labels prédits correspondent aux labels vrais pour le rapport
# Le modèle sort 'DoS_DDoS', mais le vrai label est 'DoS' ou 'DDoS'.
results['True_Label_Merged'] = results['True_Label'].replace({'DoS': 'DoS_DDoS', 'DDoS': 'DoS_DDoS'})

# Calcul des métriques globales
accuracy = accuracy_score(results['True_Label_Merged'], results['type'])
print(f"🎯 Accuracy Globale : {accuracy:.2%}")

print("\n📊 Rapport de Classification Détaillé :")
print(classification_report(results['True_Label_Merged'], results['type']))

In [ ]:
# Matrice de Confusion Visuelle
plt.figure(figsize=(12, 10))
cm = confusion_matrix(results['True_Label_Merged'], results['type'])
labels = sorted(results['True_Label_Merged'].unique())

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Matrice de Confusion (Système Hybride)')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.show()

## 3. Test de Robustesse (Simulation de Bruit)

Dans la réalité, le trafic n'est jamais aussi propre que dans le dataset CICIDS2017.
Nous allons ajouter du **bruit gaussien** aux données pour voir si le modèle tient le coup.

In [ ]:
def add_noise(X, noise_level=0.1):
    """
    Ajoute du bruit aléatoire aux features.
    noise_level: Ecart-type du bruit (proportionnel à l'écart-type de la feature)
    """
    noise = np.random.normal(0, noise_level, X.shape)
    # On multiplie par l'écart-type de chaque colonne pour que le bruit soit à l'échelle
    X_noisy = X + noise * X.std().values
    return X_noisy

# Test avec différents niveaux de bruit
noise_levels = [0.0, 0.1, 0.2, 0.5] # 0%, 10%, 20%, 50% de bruit
accuracies = []

print("🛡️ Test de Robustesse en cours...")
for nl in noise_levels:
    X_noisy = add_noise(X_test, noise_level=nl)
    res_noisy = ids.predict_df(X_noisy)
    acc = accuracy_score(results['True_Label_Merged'], res_noisy['type'])
    accuracies.append(acc)
    print(f"   Bruit {nl*100}% -> Accuracy: {acc:.2%}")

# Visualisation de la chute de performance
plt.figure(figsize=(8, 5))
plt.plot(noise_levels, accuracies, marker='o', linestyle='-', color='red')
plt.title('Robustesse du Modèle face au Bruit')
plt.xlabel('Niveau de Bruit (Noise Level)')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

## 4. Analyse des Erreurs

Où le modèle se trompe-t-il le plus ? C'est crucial pour l'amélioration.

In [ ]:
# Filtrer les erreurs
errors = results[results['True_Label_Merged'] != results['type']]

print(f"Nombre total d'erreurs : {len(errors)} sur {len(results)} prédictions.")
print("Top 5 des confusions (Vrai -> Prédit) :")
confusion_counts = errors.groupby(['True_Label_Merged', 'type']).size().sort_values(ascending=False).head(5)
print(confusion_counts)

## 5. Conclusion de l'Évaluation

**Résumé :**
- Si l'accuracy reste > 90% même avec un peu de bruit, le modèle est robuste.
- Regardez la matrice de confusion : est-ce qu'on confond souvent `DoS` et `PortScan` ? C'est moins grave que de confondre `Attaque` et `Normal`.

**Prochaine étape :** Simulation Temps Réel !